#### **Text Splitters**  
Text Splitting is the process of breaking large text files (like PDFs, books, articles) into smaller, manageable pieces (called “chunks”) so that LLMs (like GPT) can process them effectively.

**Why is it necessary?**  
LLMs have context length limits, meaning they can only handle a limited number of tokens/words at once. If you give too much text, they either fail or give poor quality output.

##### **Length Based Text Splitter**- 

Length-based splitting is the simplest and fastest method to break large text into smaller chunks by fixed size.  
Large paragraph → Split into 100-character chunks → Each chunk becomes part of a list.  

**Drawback:**   
It doesn’t consider grammar, word boundaries, or meaning.  
May split in the middle of a word or sentence, leading to broken context and less meaningful embeddings.  

For RAG-based applications:  
Chunk Overlap = 10% to 20% of chunk size (e.g., if chunk size = 100 → overlap = 10 to 20)

In [9]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path='Data Sources\\dl-curriculum.pdf')
docs = loader.load()

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separator="")

result = splitter.split_documents(docs)
print(result)

[Document(metadata={'producer': 'Skia/PDF m131 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Deep Learning Curriculum', 'source': 'Data Sources\\dl-curriculum.pdf', 'total_pages': 23, 'page': 0, 'page_label': '1'}, page_content='CampusXDeepLearningCurriculum\nA.ArtificialNeuralNetworkandhowtoimprovethem\n1.BiologicalInspiration\n● Understandingtheneuronstructure● Synapsesandsignaltransmission● Howbiologicalconceptstranslatetoartificialneurons\n2.HistoryofNeuralNetworks\n● Earlymodels(Perceptron)● BackpropagationandMLPs● The"AIWinter"andresurgenceofneuralnetworks● Emergenceofdeeplearning\n3.PerceptronandMultilayerPerceptrons(MLP)\n● Single-layerperceptronlimitations● XORproblemandtheneedforhiddenlayers● MLParchitecture\n4. LayersandTheirFunctions\n● InputLayer○ Acceptinginputdata● HiddenLayers○ Featureextraction● OutputLayer○ Producingfinalpredictions\n5.ActivationFunctions'), Document(metadata={'producer': 'Skia/PDF m131 Google Docs Renderer', 'creator': 'PyP

##### **Text Structure Based Splitting - RecursiveCharacterTextSplitting**  
This technique breaks large text into smaller chunks using the natural structure of the text: Paragraphs (\n\n), Sentences (\n), Words ( ), Characters.  
It tries to preserve meaning by avoiding mid-word or mid-sentence cuts, unlike fixed-length (length-based) splitting.

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text = """ Text is naturally organized into hierarchical units such as paragraphs, sentences, and words. We can leverage this inherent structure to inform our splitting strategy, creating split that maintain natural language flow, maintain semantic coherence within split, and adapts to varying levels of text granularity. LangChain’s RecursiveCharacterTextSplitter implements this concept:
The RecursiveCharacterTextSplitter attempts to keep larger units (e.g., paragraphs) intact.
If a unit exceeds the chunk size, it moves to the next level (e.g., sentences).
This process continues down to the word level if necessary.
"""

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
)

chunks = splitter.split_text(text)

print(len(chunks))
print(chunks)

8
['Text is naturally organized into hierarchical units such as paragraphs, sentences, and words. We', 'words. We can leverage this inherent structure to inform our splitting strategy, creating split', 'split that maintain natural language flow, maintain semantic coherence within split, and adapts to', 'adapts to varying levels of text granularity. LangChain’s RecursiveCharacterTextSplitter implements', 'this concept:', 'The RecursiveCharacterTextSplitter attempts to keep larger units (e.g., paragraphs) intact.', 'If a unit exceeds the chunk size, it moves to the next level (e.g., sentences).', 'This process continues down to the word level if necessary.']


##### **Document Structure Based Text Splitting**  
Unlike normal text (which is split using paragraphs, sentences, and words), some documents like code, markdown, or HTML are not structured in natural language. So they need to be split based on their document-specific constructs.  

Use **RecursiveCharacterTextSplitter.from_language()**- LangChain allows you to split these document types using a custom separator list for each language.  
You can apply it to: Python code, Java, JS, HTML, Markdown files, Other structured formats

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language

text = """
class Student:
    def __init__(self, name, age, grade):
        self.name = name
        self.age = age
        self.grade = grade  # Grade is a float (like 8.5 or 9.2)

    def get_details(self):
        return self.name"

    def is_passing(self):
        return self.grade >= 6.0


# Example usage
student1 = Student("Aarav", 20, 8.2)
print(student1.get_details())

if student1.is_passing():
    print("The student is passing.")
else:
    print("The student is not passing.")

"""

splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=300,
    chunk_overlap=0
)

chunks = splitter.split_text(text)
print(len(chunks))
print(chunks)

2
['class Student:\n    def __init__(self, name, age, grade):\n        self.name = name\n        self.age = age\n        self.grade = grade  # Grade is a float (like 8.5 or 9.2)\n\n    def get_details(self):\n        return self.name"\n\n    def is_passing(self):\n        return self.grade >= 6.0', '# Example usage\nstudent1 = Student("Aarav", 20, 8.2)\nprint(student1.get_details())\n\nif student1.is_passing():\n    print("The student is passing.")\nelse:\n    print("The student is not passing.")']


In [17]:
text = """
# Project Name: Smart Student Tracker

A simple Python-based project to manage and track student data, including their grades, age, and academic status.


## Features

- Add new students with relevant info
- View student details
- Check if a student is passing
- Easily extendable class-based design


## 🛠 Tech Stack

- Python 3.10+
- No external dependencies


## Getting Started

1. Clone the repo  
   ```bash
   git clone https://github.com/your-username/student-tracker.git

"""

# Initialize the splitter
splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN,
    chunk_size=200,
    chunk_overlap=0,
)

# Perform the split
chunks = splitter.split_text(text)

print(len(chunks))
print(chunks[0])

3
# Project Name: Smart Student Tracker

A simple Python-based project to manage and track student data, including their grades, age, and academic status.


##### **Semantic Meaning Based Text Splitting**  
technique splits text based on meaning, not on length or structure.  

**Why Semantic Splitting is Needed**  
It detects shifts in topic by comparing sentence meanings.  
It avoids mixing unrelated content in a single chunk.  
Useful in cases where contextual clarity is crucial for downstream tasks (like embeddings or RAG).

**Performance not very accurate**

In [24]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings

text = """
Farmers were working hard in the fields, preparing the soil and planting seeds for the next season. The sun was bright, and the air smelled of earth and fresh grass.
The Indian Premier League (IPL) is the biggest cricket league in the world. People all over the world watch the matches and cheer for their favourite teams.
Terrorism is a big danger to peace and safety. It causes harm to people and creates fear in cities and villages. When such attacks happen, they leave behind pain and sadness. To fight terrorism, we need strong laws, alert security forces, and support from people who care about peace and safety.
"""

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda"}
)

text_splitter = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type="standard_deviation",
    breakpoint_threshold_amount=0.1,
)

chunks = text_splitter.split_text(text)
print(len(chunks))
print(chunks)

3
['\nFarmers were working hard in the fields, preparing the soil and planting seeds for the next season.', 'The sun was bright, and the air smelled of earth and fresh grass. The Indian Premier League (IPL) is the biggest cricket league in the world. People all over the world watch the matches and cheer for their favourite teams. Terrorism is a big danger to peace and safety. It causes harm to people and creates fear in cities and villages. When such attacks happen, they leave behind pain and sadness. To fight terrorism, we need strong laws, alert security forces, and support from people who care about peace and safety.', '']
